In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
gloabal_prot = pd.read_csv("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_processed_data/FFPE_discovery_phosphoprotein_imputed.tsv",delimiter='\t')
gloabal_prot.head()

,uniprot,HGNC symbol,HGNC name,HGNC Location,Index,Peptide,MaxPepProb,ReferenceIntensity,FD_p01_MC_13136_42571_C1,FD_p01_FH_57_372440,...,FD_p20_MC_2685_37845_B1,FD_p20_FH_91_555175,FD_p21_UA_CU-18_S-14-0036688_B4,FD_p21_MC_3480_38147_A2,FD_p21_UA_CU-33_S-14-0021444_B4,FD_p21_MC_7203_40170_B2,FD_p21_MC_4322_38538_C14,FD_p21_UA_CU-22_S-15-0030998_A1,FD_p21_FH_20_218201,FD_p21_FH_27_228608
0,Q9NRG9,AAAS,aladin WD repeat nucleoporin,12q13.13,sp|Q9NRG9|AAAS_HUMAN_485_495_1_0,IAHIPLYFVNAQFPRFSPVLGR,1.0000,18.979822,19.116452,19.334852,...,19.162860,19.096620,19.844813,19.096951,19.140461,18.954384,19.131445,18.973132,19.289891,19.218293
1,Q9NRG9,AAAS,aladin WD repeat nucleoporin,12q13.13,sp|Q9NRG9|AAAS_HUMAN_495_495_1_1_S495,FsPVLGR,0.9968,20.733187,20.596693,19.190276,...,20.466001,18.937098,21.257011,20.201871,20.320647,20.842221,20.487372,20.921728,19.411661,19.365043
2,Q2M2I8,AAK1,AP2 associated kinase 1,2p13.3,sp|Q2M2I8|AAK1_HUMAN_605_606_1_1_T606,VQTtPPPAVQGQK,1.0000,19.971952,19.998902,19.935920,...,20.303690,19.850604,19.574052,19.815511,20.040171,19.978308,20.203371,20.035385,19.953140,20.156445
3,Q2M2I8,AAK1,AP2 associated kinase 1,2p13.3,sp|Q2M2I8|AAK1_HUMAN_618_624_1_0,VGSLTPPSSPK,1.0000,18.843687,18.930682,18.494442,...,18.993095,18.728670,18.929912,18.785851,18.746395,18.749549,18.948708,18.731892,18.650291,18.777179
4,Q2M2I8,AAK1,AP2 associated kinase 1,2p13.3,sp|Q2M2I8|AAK1_HUMAN_637_650_1_0,RILSDVTHSAVFGVPASK;ILSDVTHSAVFGVPASK,1.0000,21.232957,21.401207,21.517814,...,21.893519,21.613085,20.352800,21.253495,21.230546,21.365942,21.336141,20.996869,21.687541,21.106123


In [3]:
clinical = pd.read_excel("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_processed_data/PTRC-HGSOC_List_clincal_data.xlsx")
len(clinical["Sample ID"].unique()) # 174... samles, 159 patients... 168 genmics cols.... 
clinical.head()

,File Name,Patient ID,Sample Source,Sample ID,Image Name,Image ID,Patient Age,Patient Ethnicity,Patient Race,Neo-adjuvant,Other cancer diagnosis?,Tumor response,Tumor type,Tumor location,Tumor Location Group,Tumor Grade,Tumor Stage,Tumor Substage,Sample type
0,1009402_171069_ImageActual,2,FHCRC,205927,1009402,171069,73,Not Hispanic or Latino,White,no,NaN,refractory,Metastatic,Omentum,OM,3,III,IIIC,FFPE
1,1009403_171101_ImageActual,2,FHCRC,205927,1009403,171101,73,Not Hispanic or Latino,White,no,NaN,refractory,Metastatic,Omentum,OM,3,III,IIIC,FFPE
2,1009404_171121_ImageActual,4,FHCRC,206053,1009404,171121,60,NaN,NaN,no,NaN,sensitive,Primary,Ovary - Left,OV,2,III,IIIC,FFPE
3,1009405_416622_ImageActual,4,FHCRC,206053,1009405,416622,60,NaN,NaN,no,NaN,sensitive,Primary,Ovary - Left,OV,2,III,IIIC,FFPE
4,1009406_171004_ImageActual,7,FHCRC,206382,1009406,171004,52,Not Hispanic or Latino,NaN,no,NaN,refractory,Primary,Ovary - Left,OV,3,III,IIIC,FFPE


In [4]:
gloabal_prot_t = gloabal_prot.T

# drop middle rows .... make header the prot name... 
gloabal_prot_t.columns = gloabal_prot_t.iloc[1] + "_" + gloabal_prot_t.iloc[4] 


gloabal_prot_t = gloabal_prot_t.iloc[7:]
gloabal_prot_t.head(15)

row_to_subtract = gloabal_prot_t.iloc[0]
# Subtract this row from all other rows
df_subtracted = gloabal_prot_t.subtract(row_to_subtract, axis='columns')
df_subtracted = df_subtracted.iloc[1:]
# norm standaard scalar
scaler = StandardScaler()
# Fit and transform the DataFrame
df_subtracted = pd.DataFrame(scaler.fit_transform(df_subtracted), columns=df_subtracted.columns, index=df_subtracted.index)

df_subtracted["Sample ID1"] = df_subtracted.index

print(df_subtracted["Sample ID1"].value_counts()) # 168 patients.... but some not in other row?.,... maybe duplicates for the same patient? Or done with different divices to mearue....

# # Join with clinical now....
def custom_join(df1, df2, column1, column2):
    # Create a new DataFrame to store the result
    df1[column1] = df1[column1].astype(str)
    df2[column2] = df2[column2].astype(str)

    # Create a new DataFrame to store the result
    result_df = pd.DataFrame()

    for index, row in df1.iterrows():
        matching_rows = df2[df2[column2].apply(lambda x: x in row[column1])]
        for _, match in matching_rows.iterrows():
            result_row = pd.concat([row, match])
            result_df = result_df.append(result_row, ignore_index=True)

    return result_df


result = custom_join(df_subtracted, clinical, 'Sample ID1', 'Sample ID')

FD_p01_MC_13136_42571_C1            1
FD_p15_FH_29_232982                 1
FD_p14_UA_CU-03_S-08-0007631_A2     1
FD_p14_FH_49_296927                 1
FD_p14_FH_94_654490                 1
                                   ..
FD_p08_UA_CU-36_S-12-0032630_B14    1
FD_p08_FH_90_555596                 1
FD_p08_UA_CU-21_S-17-0012504_C1     1
FD_p08_MC_13280_42695_C3            1
FD_p21_FH_27_228608                 1
Name: Sample ID1, Length: 168, dtype: int64


In [5]:
result['label'] = result['Tumor response'].map({'sensitive': 1, 'refractory': 0, 'Sensitive': 1, 'Refractory': 0})

In [6]:
headers = ["File Name","Patient ID","label","Tumor type"]

all_prots = result.iloc[:, 0:2648].columns

prots = all_prots.to_list()

for prot in all_prots:
    headers.append(prot)

final_df = result[headers]
final_df = result.rename(columns={'File Name': 'slide_id', 'Patient ID': 'case_id'})
final_df.head()

,AAAS_sp|Q9NRG9|AAAS_HUMAN_485_495_1_0,AAAS_sp|Q9NRG9|AAAS_HUMAN_495_495_1_1_S495,AAK1_sp|Q2M2I8|AAK1_HUMAN_605_606_1_1_T606,AAK1_sp|Q2M2I8|AAK1_HUMAN_618_624_1_0,AAK1_sp|Q2M2I8|AAK1_HUMAN_637_650_1_0,AAK1_sp|Q2M2I8|AAK1_HUMAN_637_650_1_1_S637,AAK1_sp|Q2M2I8|AAK1_HUMAN_670_678_1_1_S678,ABCF1_sp|Q8NE71|ABCF1_HUMAN_105_109_1_0,ABCF1_sp|Q8NE71|ABCF1_HUMAN_105_109_1_1_S109,ABCF1_sp|Q8NE71|ABCF1_HUMAN_140_140_1_1_S140,...,Other cancer diagnosis?,Tumor response,Tumor type,Tumor location,Tumor Location Group,Tumor Grade,Tumor Stage,Tumor Substage,Sample type,label
0,-0.037443,0.489365,0.059302,0.570618,0.529554,0.753645,-0.362624,-1.063083,-1.891733,-1.441683,...,NaN,refractory,Metastatic,omental cake,OM,3.0,IV,NaN,FFPE,0
1,-0.037443,0.489365,0.059302,0.570618,0.529554,0.753645,-0.362624,-1.063083,-1.891733,-1.441683,...,NaN,refractory,Metastatic,omental cake,OM,3.0,IV,NaN,FFPE,0
2,1.020467,-1.685359,-0.210167,-1.156619,0.876121,1.114571,0.041112,2.470983,1.155111,1.359668,...,NaN,sensitive,Primary,Ovary - Left,OV,3.0,III,IIIA,FFPE,1
3,1.020467,-1.685359,-0.210167,-1.156619,0.876121,1.114571,0.041112,2.470983,1.155111,1.359668,...,NaN,sensitive,Primary,Ovary - Left,OV,3.0,III,IIIA,FFPE,1
4,-0.503854,-0.157222,-0.549445,0.188028,-1.092355,-1.360181,1.416590,-1.057596,-1.254191,-1.164753,...,NaN,sensitive,Metastatic,peritoneal biopsy,PT,3.0,III,NaN,FFPE,1


In [7]:
print(len(final_df)) #... but only 348 slides.......?
print(len(final_df["slide_id"].value_counts())) #344 unique only for phospo?...

df_unique = final_df.drop_duplicates(subset="slide_id") 

df_unique["slide_id"].value_counts()
print(len(df_unique))
# take just one from each.... 
df_unique.to_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/HGSOC_multimodal_phospho_oz_merge.csv",index=None) #check why not 348? but 344?

368
348
348


### Test the classical model that there is signal here

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, roc_auc_score


phospo_prots = pd.read_excel("/mnt/ncshare/ozkilim/BRCA/data/HGSOC_processed_data/mmc3.xlsx",sheet_name="Phospho_predictors")

phospo_prots = phospo_prots["Phospho predictors"].to_list()



In [9]:
# Define the column you want to split by
group_column = 'Sample ID'

# Create a list of unique groups
unique_groups = df_unique[group_column].unique()

# Split the unique groups into training and test sets
train_groups, test_groups = train_test_split(unique_groups, test_size=0.2, random_state=42)

# Create training and test DataFrames
train_df = df_unique[df_unique[group_column].isin(train_groups)]
test_df = df_unique[df_unique[group_column].isin(test_groups)]

# train_df, test_df  = train_test_split(result, test_size=0.2, random_state=42, stratify=result["Sample ID"])

X_train = train_df[phospo_prots]
X_test =  test_df[phospo_prots]
y_train = train_df['label']
y_test = test_df['label']

# Initialize and train the Random Forest classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

# Make predictions on the test set
rf_pred = rf_clf.predict(X_test)
rf_pred_proba = rf_clf.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

# Evaluate the Random Forest model.
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_roc_auc = roc_auc_score(y_test, rf_pred_proba)
print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest ROC AUC: {rf_roc_auc}")

Random Forest Accuracy: 0.7142857142857143
Random Forest ROC AUC: 0.835


In [10]:
# append the other df's
df_unique['label'].value_counts()

1    202
0    146
Name: label, dtype: int64

In [11]:
clinical["Tumor response"].value_counts()

sensitive     156
refractory    100
Refractory     46
Sensitive      46
Name: Tumor response, dtype: int64